In [1]:
import numpy as np
import pandas as pd

import anndata as ad
import scanpy as sc
import squidpy as sq

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as clr
from matplotlib.pyplot import rc_context
import glasbey

import scvi

c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\numba\core\decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\anndata\utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
c

In [2]:
sc.settings.n_jobs = 12

In [3]:
import torch
torch.set_float32_matmul_precision('high')
torch.cuda.is_available()

True

In [4]:
from lightning.pytorch import seed_everything
seed_everything(12345)

Seed set to 12345


12345

In [5]:
category='Immune'

In [6]:
query_genes = sc.read_h5ad("../../data/merfish/BAYSOR/seurat_objects/ns-atlas.merfish_baysor.scanvi_integrated.anndata.h5ad").var_names.tolist()

In [7]:
reference = sc.read_h5ad(f'../../data/scrna/reclustering/{category}.reclustered.seurat_object.annotated.anndata.h5ad')
reference = reference[:, query_genes].copy()

In [13]:
scvi.model.SCVI.setup_anndata(reference, batch_key="sample_barcode", layer="counts")
scvi_ref = scvi.model.SCVI(
    reference,
    use_layer_norm="both",
    use_batch_norm="none",
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2
)
scvi_ref.train(accelerator='gpu', max_epochs=25)
scvi_ref.save(f'../../data/scrna/{category}_scvi_reference_model', overwrite=True)

c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\data\fields\_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\data\fields\_dataframe_field.py:187: UserWarning: Category 42 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\model\_scvi.py:159: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Paula\miniconda

Epoch 25/25: 100%|██████████| 25/25 [02:45<00:00,  6.68s/it, v_num=1, train_loss_step=139, train_loss_epoch=130]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 25/25: 100%|██████████| 25/25 [02:45<00:00,  6.62s/it, v_num=1, train_loss_step=139, train_loss_epoch=130]


In [14]:
reference.obsm["scvi"] = scvi_ref.get_latent_representation()

In [15]:
SCANVI_LABELS_KEY = "labels_scanvi"
reference.obs[SCANVI_LABELS_KEY] = reference.obs['cell_type.detailed'].values

In [16]:
scanvi_ref = scvi.model.SCANVI.from_scvi_model(scvi_ref, labels_key=SCANVI_LABELS_KEY, unlabeled_category="UNKNOWN")
scanvi_ref.train(accelerator='gpu', max_epochs=20, n_samples_per_label=100)
scanvi_ref.save(f'../../data/scrna/{category}_scanvi_reference_model', overwrite=True)

c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\data\fields\_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\data\fields\_dataframe_field.py:187: UserWarning: Category 42 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(


INFO     Training for 20 epochs.                                                                                   


c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\scvi\model\_scanvi.py:139: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(self.adata_manager, n_batch)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Paula\miniconda3\envs\rapids\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 20/20: 100%|██████████| 20/20 [04:26<00:00, 12.51s/it, v_num=1, train_loss_step=127, train_loss_epoch=132]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 20/20: 100%|██████████| 20/20 [04:26<00:00, 13.34s/it, v_num=1, train_loss_step=127, train_loss_epoch=132]


In [17]:
reference.obsm["scanvi"] = scanvi_ref.get_latent_representation()

In [18]:
reference.write_h5ad(f'../../data/scrna/{category}.reclustered.annotated.scanvi_reference.anndata.h5ad')